In [1]:

import zstandard
import json
from datetime import datetime

import pandas as pd


years = ["2019"]
months = ["07","08","09","10", "11","12"]

for y in years:
    for m in months:
        year_month = f"{y}-{m}"

        non_dep_cohort = pd.read_csv(f"{year_month}-non-dep.csv")
        non_dep_cohort = non_dep_cohort[["author", "created_utc",  "is_self", "subreddit", "subreddit_subscribers", "title", "selftext"]]
        non_dep_cohort = non_dep_cohort.loc[~non_dep_cohort["author"].isin(["[deleted]","AutoModerator"])] 
        non_dep_users = non_dep_cohort["author"].unique()

        print(len(non_dep_users))

        nondep_data = []
        with open(f"RS_{year_month}.zst", 'rb') as fh:
            print(f"RS_{year_month}.zst")
            dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
            with dctx.stream_reader(fh) as reader:
                previous_line = ""
                while True:
                    chunk = reader.read(2**24)  # 16mb chunks
                    if not chunk:
                        break

                    string_data = chunk.decode('utf-8')
                    lines = string_data.split("\n")
                    for i, line in enumerate(lines[:-1]):
                        if i == 0:
                            line = previous_line + line
                        object = json.loads(line)

                        if object["is_self"]:
                            if object["author"] in non_dep_users:
                                nondep_data.append(object)
                    previous_line = lines[-1]
        
        pd.DataFrame(nondep_data).to_csv(f"{year_month}-non-dep-data.csv",index=False)

800
RS_2019-07.zst
877
RS_2019-08.zst
633
RS_2019-09.zst
624
RS_2019-10.zst
606
RS_2019-11.zst
691
RS_2019-12.zst


,archived,author,author_created_utc,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status,wls,media_metadata,author_cakeday
0,False,AnimeMod,1329103241,None,None,[],None,None,None,text,...,new,self,NaN,NaN,Recommendation Tuesdays Megathread - Week of J...,https://www.reddit.com/r/anime/comments/abcug1...,all_ads,6.0,NaN,NaN
1,False,PossibleParty,1540591650,None,None,[],None,None,None,text,...,None,self,NaN,NaN,[GEAR] Can anyone tell me what this guitar is?...,https://www.reddit.com/r/Guitar/comments/abcug...,all_ads,6.0,NaN,NaN
2,False,SonOfMan6,1546295388,None,None,[],None,None,None,text,...,None,default,NaN,NaN,HERE IS WISDOM. LET HE THAT HATH UNDERSTANDING...,https://www.reddit.com/r/confession/comments/a...,house_only,1.0,NaN,NaN
3,False,DwellingInKaruna,1499882661,None,None,[],None,None,None,text,...,None,self,NaN,NaN,Manga with martial arts you could probably lea...,https://www.reddit.com/r/manga/comments/abcuid...,all_ads,6.0,NaN,NaN
4,False,RotenkstheSayain,1472770053,None,None,[],None,None,None,text,...,None,self,NaN,NaN,Mickey Mouse has never actually eaten cheese,https://www.reddit.com/r/Showerthoughts/commen...,all_ads,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,False,vincent365,1488583105,None,None,[],None,None,None,text,...,None,self,NaN,NaN,"What are some new year, goals that sound great...",https://www.reddit.com/r/AskReddit/comments/ab...,all_ads,6.0,NaN,NaN
1005,False,ANIME_XAN,1543201077,None,None,[],None,None,None,text,...,None,self,NaN,NaN,I Just want to talk to people.,https://www.reddit.com/r/friendship/comments/a...,None,NaN,NaN,NaN
1006,False,timberician,1539926573,,,[],96bf97d4-8cea-11e2-b2a6-12313d18803b,¯\_(ツ)_/¯,dark,text,...,None,self,NaN,NaN,I hate New Years,https://www.reddit.com/r/raisedbynarcissists/c...,no_ads,0.0,NaN,NaN
1007,False,MyUsernameIsFree,1528736866,None,None,[],None,None,None,text,...,None,self,NaN,NaN,Best place to open a bank account? And how?,https://www.reddit.com/r/internetparents/comme...,None,NaN,NaN,NaN


In [10]:
datetime.fromtimestamp(object["created_utc"])

datetime.datetime(2019, 1, 1, 1, 26, 9)